# 📚 LangChain ile Retrieval Augmented Generation'a Giriş 🦜🔗

Bu not defterinde LangChain kullanarak Retrieval Augmented Generation'ı nasıl kullanacağınızı öğreneceksiniz.

Kendi belgelerimiz hakkında sorular sormak için bir LLM kullanacağız!

## ⚙️ Kurulum

👉 Temel kütüphaneleri içe aktarmak için aşağıdaki hücreyi çalıştırın.

In [1]:
%load_ext autoreload
%autoreload 2
import os
from pprint import pprint
from IPython.display import Markdown

👉 API anahtarımızı tekrar yüklemek için aşağıdaki hücreyi çalıştırın:

In [2]:
from dotenv import load_dotenv

load_dotenv()  # Load environment variables from .env file

True

## 📚 Neden RAG?

Bir LLM kendi başına, öğrendiği her şey hakkında sorulara yanıt verebilir.

Bunun birkaç dezavantajı vardır:
- Eğitim verileri geçmişten gelir ve en son verilerle güncellenmez.
- Sadece eğitim aldığı verileri bilir.

Bir LLM'yi kendi verilerimizle çalışması için kullanmak istiyoruz. İşte bu noktada RAG (Retrieval-Augmented Generation) devreye girer.

1. **Retrieval-Augmented Generation (RAG)**, gerçek doğruluğu artırmak için bir dil modelini belge alıcı ile birleştirir.
2. **İlgili dış belgeleri alır** (örneğin, bilgi tabanından) yanıtlar üretmeden önce.
3. **Dil modeli hem istemi hem de alınan bağlamı kullanarak** daha bilgili ve temelli çıktılar üretir.

## 🇪🇺 Bağlam

Bu meydan okumada, Avrupa Parlamentosu'ndan belgelerle çalışacağız.

Bir gazeteci olduğunuzu ve Avrupa Parlamentosu'nun genel kurul oturumları sırasında belirli bir konu hakkında neler söylendiğini öğrenmek istediğinizi düşünün. Bu oturumlar yılda 12 kez Strasbourg'da gerçekleşir ve 4 gün sürer. Oturumların transkriptleri EP'nin web sitesinde mevcuttur.

Kesinlikle tüm bu transkriptleri karıştırmak istemezsiniz. O halde, hayatımızı kolaylaştırmak için RAG'ı kullanalım!

Bu, her zaman test etmek için yepyeni veriler alabileceğimiz için çalışmak üzere iyi verilerdir.

## 📘 Verileri alalım

1. [EP'nin web sitesine](https://www.europarl.europa.eu/plenary/en/debates-video.html) gidin. 
1. Bu sizi en son genel kurul oturumuna yönlendirecektir.
1. İlk tarihin altında, "▶️ Verbatim reports HTML" bölümünde `HTML`'e tıklayın.
1. Sayfanın sonuna kaydırın ve alttaki PDF dosyasını indirin.
1. Dosyayı `data` klasörüne kaydedin.

Bir belgeyle başlayacağız, ancak daha sonrası için diğer birkaç günün aynısını şimdiden indirebilirsiniz.

Belgeye bir göz atın. Kaç sayfası var? Belge hakkında bir fikir edinmek için hızlıca belgede gezinin.

## 🔢 Belgeleri gömme

Belgeleri gömmek, tüm belgeleri veya belge parçalarını vektörlere çevirmek anlamına gelir.

LangChain🦜🔗 yine çok yardımcı olacak.

Bir gömme aracı (embedder) başlatalım ve deneyelim. LLM olarak Gemini kullandığımız için, Google'ın metin gömme araçlarında kalalım.

In [3]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

👉 Basit bir metin parçasını gömmek için gömme aracının `.embed_query()` metodunu deneyin.

In [4]:
# Embed a text like "What is the capital of France?" and save it to a variable `sample_embedding`

sample_embedding = embeddings.embed_query("What is the capital of France?")


👉 Bu `sample_embedding`'i keşfetmek için zaman ayırın. Nasıl görünüyor? Tipi nedir? Gömme boyutu nedir?

In [5]:
print(type(sample_embedding))
print(len(sample_embedding))

<class 'list'>
3072


## 💾 PDF'den gerçek verilerimizi yükle

Artık bir gömmenin nasıl göründüğünü biliyoruz, gerçek verilerimizle çalışmanın zamanı geldi.

👉 [LangChain belgelerine](https://docs.langchain.com/oss/python/integrations/document_loaders/index#pdfs) gidin ve PyPDF kullanarak bir PDF'yi nasıl yükleyebileceğinizi öğrenin.

👉 Sonra devam edin ve daha önce indirdiğiniz PDF'lerden birini yükleyin.

In [6]:
!curl https://www.europarl.europa.eu/doceo/document/CRE-10-2025-05-05_EN.pdf > data/CRE-10-2025-05-05_EN.pdf
!curl https://www.europarl.europa.eu/doceo/document/CRE-10-2025-05-06_EN.pdf > data/CRE-10-2025-05-06_EN.pdf

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  835k  100  835k    0     0   522k      0  0:00:01  0:00:01 --:--:--  522k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1633k  100 1633k    0     0  2689k      0 --:--:-- --:--:-- --:--:-- 2686k


👉 `pages`'i keşfedin:
- Veri tipi nedir?
- Kaç sayfanız var?
- Bir sayfanın tipi nedir?
- Bir sayfanın içeriğine nasıl erişebilirsiniz?
- Tam belgenin kaç karakteri var?
- Bir sayfanın `metadata`'sında neler var?

In [7]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "data/CRE-10-2025-05-05_EN.pdf"

loader = PyPDFLoader(file_path)

pages = loader.load()

In [8]:
print("Type of pages:          ", type(pages))
print("Nb of pages:            ", len(pages))
print("Type of one page:       ", type(pages[0]))
print("Total nb of characters: ", sum([len(page.page_content) for page in pages]))

Type of pages:           <class 'list'>
Nb of pages:             106
Type of one page:        <class 'langchain_core.documents.base.Document'>
Total nb of characters:  340488


In [9]:
Markdown(pages[40].page_content)

05-05-2025 41
As no one wishes to speak against, Mr Geadi, do you agree with the alternative proposal from the 
S&D Group, on the title and the resolution?
1-0073-0000
Γεάδης Γεάδη, εξ ονόματος της ομάδας ECR. – Κυρία Πρόεδρε, σε πνεύμα εποικοδομητικής 
συνεργασίας, που οφείλουμε όλοι να επιδεικνύουμε, είμαστε πρόθυμοι να αποδεχθούμε την 
τροποποίηση που προτείνει η Ομάδα S&D για συντόμευση του τίτλου της πρότασης που έχουμε 
υποβάλει. Χαιρόμαστε που η εισήγηση βρίσκει στήριξη και καλλιεργεί πνεύμα ενότητας απέναντι στις 
απαράδεκτες τουρκικές απειλές.
Επιπλέον, παρακαλώ όπως τεθεί σε ονομαστική ψηφοφορία το αίτημά μας να κατατεθεί σχετικό 
ψήφισμα επί του θέματος. Θεωρούμε ότι, πέρα από τη συζήτηση, ένα ψήφισμα από τα μέλη του 
Κοινοβουλίου θα στείλει ισχυρό μήνυμα στήριξης της Κυπριακής Δημοκρατίας και ότι η Ευρωπαϊκή 
Ένωση δεν συμβιβάζεται με την καταπάτηση του διεθνούς δικαίου.
1-0074-0000
President. – So first we will vote on having the debate with the title as amended by the S&D 
Group...
Yes, you can have the floor, Mr Mavrides.
1-0075-0000
Costas Mavrides, on behalf of the S&D Group. – Madam President, although in principle we agree 
with the facts as stated by my colleague, I'd like to say that, on behalf of the S&D group, we have 
proposed an alternative which has already been accepted. However, the title would be 'the illegal 
visit of President Erdoğan to the occupied areas of the Republic of Cyprus' with one round group 
of speakers without resolution. However, I'd like to announce the following: on behalf of my 
group, I also announce our strong commitment to table an oral amendment by the rapporteur on 
the 2023 and 2024 Commission reports on Turkey that would take place during the voting on 
Wednesday, and of course we expect to have the full support of this House.
1-0076-0000
President. – OK, so let me get this clear. We're going to vote on the debate with the title as 
amended by the S&D Group which was accepted by the ECR Group. What is not clear to me is 
whether the S&D would want the debate on Wednesday or on Thursday. You say Wednesday? 
OK, Wednesday. Fine. We'll do it on Wednesday. We just add to our debates on Wednesday.
So we vote first by roll call on adding the statements.
(Parliament approved the request)
Now we vote by roll call on whether we will have a resolution.
(Parliament rejected the request)
We will see with Mr Mavrides what he meant and how we can do it.
Thank you very much. The agenda is adopted. Have a good week.

## ✂️ Verilerimizi böl

Tam belgemiz gömülmek için çok uzun. Metin gömme aracımız 2.048 tokena kadar giriş alabilir. Gemini modelleri için bu yaklaşık 8.196 karakterdir (token başına 4 karakter).

Bu yüzden belgemizi daha küçük parçalara bölmek istiyoruz.

Zaten çalışabileceğimiz bir dizi sayfamız var. Ama sayfa sonları biraz keyfi: genellikle cümlenin ortasında görünürler.

Ayrıca, sayfalar arasında örtüşme yoktur. Bu yüzden bir sayfanın ilk satırı önceki tüm bağlamı kaçırır. Tam metni biraz örtüşmeyle bölmek daha iyidir.

İlk olarak, PDF'yi tekrar yükleyeceğiz, bu sefer bölmeden.

In [10]:
loader = PyPDFLoader(file_path, mode='single')
pdf = loader.load()
pdf_text = pdf[0].page_content
len(pdf_text)

340698

Artık tüm PDF'imizi tek bir belge olarak aldığımıza göre, onu daha akıllı bir şekilde parçalara bölebiliriz.

👉 Yine, ["Özyinelemeli olarak bölme" konusundaki LangChain belgelerine](https://docs.langchain.com/oss/python/integrations/splitters/recursive_text_splitter) gidin ve `pdf` _belgelerimizi_ parçalara (LangChain'de `documents` olarak adlandırılır) nasıl böleceğinizi öğrenin.

2_000 karakter (bizim durumumuzda yaklaşık yarım sayfa) parçalara 400 örtüşmeyle bölün. İsterseniz diğer değerlerle deneyebilirsiniz.

`RecursiveCharacterTextSplitter`'ın `.split_documents()` metodunu kullanın: bu metod giriş olarak bir belge alır ve bölünmüş belgeler çıktısı verir.

In [11]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2_000,     
    chunk_overlap=400,     
    add_start_index=True,  
)

all_splits = text_splitter.split_documents(pdf)

print(f"Split transcript into {len(all_splits)} sub-documents.")

Split transcript into 214 sub-documents.


👉 `all_splits`'i inceleyin:
- Veri tipi nedir?
- Kaç bölümünüz var?
- Bir bölümün tipi nedir?
- Bir bölümün içeriğine nasıl erişebilirsiniz?
- Şimdi toplamda kaç karakterimiz var?
- Bir bölümün `metadata`'sında neler var?

In [12]:
print("Type of all_splits:          ", type(all_splits))
print("Nb of all_splits:            ", len(all_splits))
print("Type of one all_split:       ", type(all_splits[0]))
print("Total nb of characters:      ", sum([len(all_split.page_content) for all_split in all_splits]))

Type of all_splits:           <class 'list'>
Nb of all_splits:             214
Type of one all_split:        <class 'langchain_core.documents.base.Document'>
Total nb of characters:       416147


In [13]:
Markdown(all_splits[80].page_content)

striving to exploit the full potential of the EU's relations with the United Kingdom.
Last March, the Council exchanged views on the state of play. The upcoming first EU‑UK summit 
will provide a unique opportunity to strengthen our relationship. We are like‑minded partners, 
allies and good neighbours. Therefore, we are very much welcoming the EU governments' 
approach, seeking to further strengthen our relations.
We work together from sanctions against Russia to support for Ukraine through security summits 
and joint diplomatic efforts. The ongoing Russian aggression against Ukraine, and our joint 
support for Ukraine, is a strong reminder of why our unity matters more than ever.
At the summit, we will seek to reaffirm our mutual commitment to the full, faithful and timely 
implementation of our agreements, including rights of our citizens. At the same time, there is still 
untapped potential and room for improvement in our relations. Ahead of the upcoming EU‑UK 
summit, the Council presidencies work closely with the Commission to identify and explore areas 
for deepening our cooperation.
A whole range of areas will be discussed with our British hosts during the summit: security and 
defence; sanitary and phytosanitary rules for agricultural products; stronger cooperation on 
energy; access to waters for EU fishermen; and opportunities for young people to live, work and 
study across the border. Together we are working on a package in key areas that will bring 
tangible benefits to citizens and businesses on both sides of the Channel. Let me stress that our 
partnership is about more than just trade flows: it's about people.
Madam President, honourable Members, Commissioner, we should not forget about some 
challenges that remain. The situation in Northern Ireland requires careful monitoring, as does the 
situation of Union citizens that live in the United Kingdom.
In the relations with the UK, we are following the principles, among which there are the

In [14]:
all_splits[80].metadata

{'producer': 'Aspose.Words for Java 24.1.0',
 'creator': 'Aspose.Words',
 'creationdate': '',
 'source': 'data/CRE-10-2025-05-05_EN.pdf',
 'total_pages': 106,
 'start_index': 127446}

## 🗄️ Her şeyi bir araya getir: belgelerimizi gömme ve vektör deposunda sakla

Elimizde şunlar var:
- Bir gömme aracı
- Veriyi yüklemek için bir yükleyici
- Belgemizi belgelere bölmek için bir metin bölücü

Neyi kaçırıyoruz?

Belgelerimizi gömebiliriz, ama onları bir yerde saklamak istiyoruz. İşte burada vektör deposu devreye girer: şunları saklamamıza olanak sağlar:
- belgeyi (parçayı),
- onun gömmesini,
- meta verilerini.

Sonraki adımda belgeleri verimli bir şekilde alabilecek olacağız.

👉 Bir `InMemoryVectorStore` nasıl oluşturabileceğinizi görmek için ["Vektör depoları" üzerine LangChain belgelerini](https://docs.langchain.com/oss/python/langchain/knowledge-base#3-vector-stores) kontrol edin.

In [16]:
# Import the necessary libraries

from langchain_core.vectorstores import InMemoryVectorStore

# Create an in-memory vector store using the embedder `embeddings` we created earlier
vector_store = InMemoryVectorStore(embeddings)

# Add the `all_splits` to the vector store and store the result in a variable called `document_ids`

document_ids = vector_store.add_documents(documents=all_splits)


In [17]:
# Have a look at the first 3 document IDs
print(document_ids[:3])


['174afa13-1395-4eb8-98e6-fec43a3a5af0', 'c988943e-8e6e-495c-9a40-9877173c6a34', '47707085-5354-4a59-b4fc-0aed98927441']


In [18]:
# Use the vector store's `get_by_ids` method. You have to give it a list of document IDs.

vector_store.get_by_ids(document_ids[:3])


[Document(id='174afa13-1395-4eb8-98e6-fec43a3a5af0', metadata={'producer': 'Aspose.Words for Java 24.1.0', 'creator': 'Aspose.Words', 'creationdate': '', 'source': 'data/CRE-10-2025-05-05_EN.pdf', 'total_pages': 106, 'start_index': 0}, page_content='2024-2029\nПЪЛЕН ПРОТОКОЛ НА РАЗИСКВАНИЯТА DEBAŠU STENOGRAMMA\nACTA LITERAL DE LOS DEBATES POSĖDŽIO STENOGRAMA\nDOSLOVNÝ ZÁZNAM ZE ZASEDÁNÍ AZ ÜLÉSEK SZÓ SZERINTI JEGYZŐKÖNYVE\nFULDSTÆNDIGT FORHANDLINGSREFERAT RAPPORTI VERBATIM TAD-DIBATTITI\nAUSFÜHRLICHE SITZUNGSBERICHTE VOLLEDIG VERSLAG VAN DE VERGADERINGEN\nISTUNGI STENOGRAMM PEŁNE SPRAWOZDANIE Z OBRAD\nΠΛΗΡΗ ΠΡΑΚΤΙΚΑ ΤΩΝ ΣΥΖΗΤΗΣΕΩΝ RELATO INTEGRAL DOS DEBATES\nVERBATIM REPORT OF PROCEEDINGS STENOGRAMA DEZBATERILOR\nCOMPTE RENDU IN EXTENSO DES DÉBATS DOSLOVNÝ ZÁPIS Z ROZPRÁV\nTUARASCÁIL FOCAL AR FHOCAL NA N-IMEACHTAÍ DOBESEDNI ZAPISI RAZPRAV\nDOSLOVNO IZVJEŠĆE SANATARKAT ISTUNTOSELOSTUKSET\nRESOCONTO INTEGRALE DELLE DISCUSSIONI FULLSTÄNDIGT FÖRHANDLINGSREFERAT\nПонеделник - lunes - Pondě

👉 Bir vektör deposundaki belgenin içeriğine ve meta verilerine nasıl erişebilirsiniz?

In [19]:
one_doc = vector_store.get_by_ids(document_ids[:1])[0]
display(Markdown(one_doc.page_content))
display(one_doc.metadata)

2024-2029
ПЪЛЕН ПРОТОКОЛ НА РАЗИСКВАНИЯТА DEBAŠU STENOGRAMMA
ACTA LITERAL DE LOS DEBATES POSĖDŽIO STENOGRAMA
DOSLOVNÝ ZÁZNAM ZE ZASEDÁNÍ AZ ÜLÉSEK SZÓ SZERINTI JEGYZŐKÖNYVE
FULDSTÆNDIGT FORHANDLINGSREFERAT RAPPORTI VERBATIM TAD-DIBATTITI
AUSFÜHRLICHE SITZUNGSBERICHTE VOLLEDIG VERSLAG VAN DE VERGADERINGEN
ISTUNGI STENOGRAMM PEŁNE SPRAWOZDANIE Z OBRAD
ΠΛΗΡΗ ΠΡΑΚΤΙΚΑ ΤΩΝ ΣΥΖΗΤΗΣΕΩΝ RELATO INTEGRAL DOS DEBATES
VERBATIM REPORT OF PROCEEDINGS STENOGRAMA DEZBATERILOR
COMPTE RENDU IN EXTENSO DES DÉBATS DOSLOVNÝ ZÁPIS Z ROZPRÁV
TUARASCÁIL FOCAL AR FHOCAL NA N-IMEACHTAÍ DOBESEDNI ZAPISI RAZPRAV
DOSLOVNO IZVJEŠĆE SANATARKAT ISTUNTOSELOSTUKSET
RESOCONTO INTEGRALE DELLE DISCUSSIONI FULLSTÄNDIGT FÖRHANDLINGSREFERAT
Понеделник - lunes - Pondělí - mandag - Montag - esmaspäev - Δευτέρα - Monday
lundi - Dé Luain - ponedjeljak - lunedì - pirmdiena - Pirmadienis - hétfő - It-Tnejn
maandag - poniedziałek - Segunda-feira - luni - Pondelok - Ponedeljek - maanantai - måndag
05.05.2025
Единство в многообразието - Unida en la diversidad - Jednotná v rozmanitosti - Forenet i mangfoldighed - In Vielfalt geeint - Ühinenud mitmekesisuses
Eνωμένη στην πολυμορφία - United in diversity - Unie dans la diversité - Aontaithe san éagsúlacht - Ujedinjena u raznolikosti - Unita nella diversità
Vienoti daudzveidībā - Suvienijusi įvairovę - Egyesülve a sokféleségben - Magħquda fid-diversità - In verscheidenheid verenigd - Zjednoczona w różnorodności
Unida na diversidade - Unită în diversitate - Zjednotení v rozmanitosti - Združena v raznolikosti - Moninaisuudessaan yhtenäinen - Förenade i mångfalden
Редактирана версия - Edición revisada - Revidované vydání - Revideret udgave - Überprüfte Ausgabe - Uuendatud versioon
Αναθεωρημένη έκδοση - Revised edition - Edition révisée - Eagrán athbhreithnithe - Revidirano izdanje - Edizione rivista
Pārskatītā redakcija - Atnaujinta informacija - Lektorált változat - Edizzjoni riveduta - Herziene uitgave - Wersja poprawiona

{'producer': 'Aspose.Words for Java 24.1.0',
 'creator': 'Aspose.Words',
 'creationdate': '',
 'source': 'data/CRE-10-2025-05-05_EN.pdf',
 'total_pages': 106,
 'start_index': 0}

## 🔎 Benzer belgeleri almak için vektör deposunu kullan

Artık belgeleri gömleğe çevirdiğimize göre, benzer belgeleri almak için vektör deposunu kullanabiliriz.

👉 Bunun nasıl çalıştığını görmek için ["Vektör depoları" üzerine LangChain belgelerini](https://docs.langchain.com/oss/python/langchain/knowledge-base#3-vector-stores) kontrol edin.

Bir sorgu kullanın, örneğin "Tarım politikası üzerine tartışmayı özetle.", ve en benzer belgeleri bulun. Ayrıca alınacak belge sayısını da belirtebilirsiniz.

In [20]:
# Save your question into a variable called `query`

query = "Summarize the discussion on agricultural policy."
# Use the vector store to find similar documents to the query. Store the result in a variable called `retrieved_docs`

retrieved_docs = vector_store.similarity_search(query)

In [21]:
for doc in retrieved_docs:
    display(Markdown(doc.page_content))

every applicant, whether in the private sector or in the public sector, to bring about a project 
which really gives a return to European taxpayers.
I noticed very well the remarks on small- and medium-sized enterprises, but also micro 
businesses, and I fully agree the access to credit for these companies, these very small companies, 
who are so important when it comes to the labour market inside the EU, is still an issue we really 
have to worry about and work on, and that's what we are doing as the EIB group. We cannot do 
this directly with SMEs and micro businesses in Europe. We always go through a financial 
intermediary, mostly European commercial banks – a very important element of our business.
I listened very carefully to the remarks on agriculture, and especially young farmers receive our 
attention when it comes to the area of agriculture. For this year, we envisage to invest at least 
EUR 3 billion in this area.
In the area of housing, which was also mentioned by honourable Members, we are trying to 
leverage the financing we are going to make available to a couple of billion euro, hopefully in a 
couple of years, to EUR 300 billion annually. We have three priorities in the area of housing: one – 
innovation, supporting innovative building technologies like modular housing to make 
construction faster, cheaper and easier; second – sustainability, scaling up energy efficient 
renovation to reduce living costs when it comes to energy prices; and three – affordability, 
strengthening support for public investment tailored to the specific needs of each country and 
piloting private investments.
Now, on the issue of climate, which is also close to a bit more than half of what we are doing 
annually. This is about climate adaptation; this is about dealing with droughts, it is about dealing 
with floods – we have seen both inside many countries of the European Union, and they require

dicho nada del hundimiento del barco ni de los más de mil asesinados –cooperantes, médicos y 
enfermeras– que intentan salvar vidas. Esas muertes son una mancha indeleble en la conciencia de 
los líderes europeos que siguen cooperando con el genocidio en Gaza.
Nuestra obligación como ciudadanos es movilizarnos como garantes del Derecho internacional y 
recordar que, cuando la barbarie se normaliza, la desobediencia es una obligación moral.
1-0232-0000
Kathleen Funchion (The Left). – A Uachtaráin, ba mhaith liom an comhaontú idir an tAontas 
Eorpach agus an Iorua a ardú. Níl an comhaontú seo go maith do phobal iascaireachta na 
hÉireann agus is mór an náire é aon Fheisire as Éirinn vótáil ina fhabhar. Tá ár n-iascairí i dteideal 
córas cothrom agus ba chóir dúinn oibriú ina threo sin. Táimid i Sinn Féin tiomanta chun seasamh 
suas ar son ár bpobal cois cósta agus iascaireachta chun a chinntiú go mbeidh todhchaí bhreá agus 
gheal acu.
1-0233-0000
Γεώργιος Αυτιάς (PPE). – Κυρία Πρόεδρε, κύριε Επίτροπε, η άσκηση οικονομικής πολιτικής, πέραν της 
ανταγωνιστικότητας και της σταθερότητας —και το ξέρετε πολύ καλά αυτό, γιατί η πατρίδα μου πέρασε 
από τρία μνημόνια— πρέπει να έχει και έντονο κοινωνικό χαρακτήρα, δηλαδή στήριξη μισθών και 
συντάξεων, στήριξη φορολογικών ελαφρύνσεων, λύση δημογραφικού, στέγη. Το ξέρετε πολύ καλά, 
κύριε Επίτροπε, το θέμα, και εσείς, αξιότιμοι συνάδελφοι. Μείωση της ανεργίας και φθηνή ενέργεια.
102 05-05-2025
Προς αυτή την κατεύθυνση, η χώρα μου κινείται με ταχύτατο ρυθμό, αποπληρώνει δάνεια δεκαετίες 
μπροστά, έχει άριστες κριτικές από οίκους αξιολόγησης και, παράλληλα, πλεόνασμα. Αυτό το 
πλεόνασμα, λοιπόν, επιστρέφεται στην κοινωνία.
Να ξέρετε, κύριε Επίτροπε, ότι αυτός ο βηματισμός θα συνεχιστεί και το επόμενο χρονικό διάστημα και 
προς αυτή την κατεύθυνση σας ενημερώνω συνεχώς.
1-0234-0000
Juan Fernando López Aguilar (S&D). – Señora presidenta, vicepresidente Fitto, usted estuvo

Koninkrijk en de Europese Unie is de Overeenkomst inzake sanitaire en fytosanitaire maatregelen 
(SPS-Overeenkomst) over voedselveiligheid en grenscontroles op levensmiddelen een van de 
belangrijkste dossiers. Mijn land Vlaanderen voert jaarlijks voor 4,8 miljard EUR aan 
landbouwproducten uit naar het Verenigd Koninkrijk. Deze handel wordt echter vertraagd door 
omslachtige procedures, extra controles en uiteenlopende regels, met alle gevolgen van dien. Onze 
boeren en bedrijven verdienen geen onzekerheid, maar duidelijke afspraken. Daarom vraag ik: 
stop met treuzelen, stop met eindeloos uitstellen. We hebben een werkbaar akkoord nodig dat 
handel mogelijk maakt, zonder extra lasten en zonder toegevingen op het gebied van kwaliteit. 
We hebben behoefte aan minder bureaucratie, meer voorspelbaarheid en meer handel. Onze 
boeren hebben recht op een vlotte export en eerlijke toegang tot de markt.
1-0101-0000
Bert-Jan Ruissen (ECR). – Voorzitter, "Beter een goede buur dan een verre vriend" – dat is de 
benadering die we moeten volgen in onze betrekkingen met het Verenigd Koninkrijk. De top van 
19 mei biedt een mooie kans om hier invulling aan te geven. Het is belangrijk om het "chagrijn" 
over de Brexit achter ons te laten. Laten we investeren in onze onderlinge relaties, de handel 
bevorderen en gezamenlijk inspanningen leveren rond defensie. Dit is absoluut geen luxe in deze 
tijden van geopolitieke spanningen.
Een belangrijk aandachtspunt is het gezamenlijk beheer van de visbestanden, met name in de 
Noordzee. Voor onze vissers is het daarbij van cruciaal belang om ook na 2026 de toegang tot de 
Britse wateren te behouden. We moeten voorkomen dat zij opnieuw quota moeten inleveren. Dit 
onderwerp verdient alle aandacht, ook tijdens de komende top. Als goede buren moeten we toch 
in staat zijn om hier goede afspraken over te maken.
1-0102-0000
Nathalie Loiseau (Renew). – Madam President, dear British friends. The EU-UK summit gives us

wzmacniać mosty, modernizować drogi dojazdowe czy budować je w takich parametrach, żeby 
mogły również służyć celom obronnym. I to nie jest militaryzowanie polityki spójności, ale danie 
możliwości tym regionom, które czują taką potrzebę, realizowania tych celów.
Szanowni państwo, panie komisarzu, bardzo serdecznie dziękuję za te dzisiejsze wystąpienia. 
Dziękuję za współpracę. Mam głębokie przekonanie, że ten dokument, który w czwartek 
przegłosujemy, również pomoże panu, bowiem znamy pana historię zawodową. Wiemy, że jest 
pan samorządowcem. Był pan szefem regionu, ministrem odpowiedzialnym również za politykę 
regionalną, więc wiemy, że rozumie pan potrzeby regionu, potrzeby społeczności lokalnych. Ale 
u nas w Polsce się mówi, że diabeł tkwi w szczegółach. Co do głównych założeń polityki 
spójności zgadzamy się również, że trzeba iść w kierunku modernizacji, ewolucji, nie rewolucji. 
Ale będziemy dyskutować na temat tego, jak to w praktyce ma wyglądać i jak Komisja Europejska 
to widzi. Mam nadzieję, że wspólnie osiągniemy sukces.
1-0214-0000
Presidente. – La discussione è chiusa.
La votazione si svolgerà giovedì.
Dichiarazioni scritte (articolo 178)
1-0214-5000
Dan-Ştefan Motreanu (PPE), în scris. – Doresc să transmit un mesaj clar Comisiei Europene: nu 
avem nevoie de o centralizare a politicii de coeziune și a politicii agricole comune, după modelul 
NextGenerationEU. Dimpotrivă, este nevoie de mai multă descentralizare către autoritățile 
regionale și locale, pentru că deciziile luate mai aproape de cetățeni sunt mai bine adaptate 
realităților din teren, mai eficiente și rapide.
Într-un climat geopolitic tot mai tensionat, reamintesc că politica de coeziune este cel mai vizibil și 
concret instrument al UE pentru majoritatea cetățenilor. Investițiile în spitale, infrastructură, 
educație și IMM-uri consolidează încrederea în proiectul european și combat eficient discursurile 
eurosceptice.
96 05-05-2025

Bu, RAG'ın sözde "Alma" (Retrieval) kısmını tamamlar: artık sorgumuza en benzer belgeleri bulabiliriz.

Çalışmanın çoğu artık tamamlandı!

## 💬 Sorumuza bir cevap üret

Şimdiye kadar benzer belgeleri almamızı sağlamak için sadece bir **gömme modeli** kullandık.

Şimdi, sorumuzla bir cevap almak için üretici bir LLM kullanacağız: ona aldığımız belgeler ve sorumuzla besleyeceğiz.

Bunu yapmanın en temel yolu tüm girdilerimizi birbirine bağlamak, sorumuzla eklemek ve sonucu görmek olacaktır.

Bir deneyelim.

👉 İlk olarak önceki meydan okumalarda olduğu gibi bir LLM başlatın.

In [22]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gemini-2.5-flash-lite", model_provider="google_genai")

Sonra temel bir istem oluşturun:

In [23]:
prompt = '\n\n'.join([doc.page_content for doc in retrieved_docs])
prompt += "\n\n" + query

👉 Şimdi istemi kullanın:

In [24]:
response = llm.invoke(prompt)
Markdown(response.content)

The discussion on agricultural policy touches on several key areas:

*   **Support for Farmers, Especially Young Farmers:** There is a clear emphasis on supporting farmers, with a specific mention of "young farmers" receiving particular attention in the agricultural sector. The EIB group plans to invest at least EUR 3 billion in this area for the current year.
*   **Access to Credit for SMEs and Micro Businesses:** While not exclusively agricultural, the broader issue of access to credit for Small and Medium-sized Enterprises (SMEs) and micro businesses is highlighted. These entities are recognized as crucial for the EU's labor market. The EIB group acknowledges that they work through financial intermediaries, primarily European commercial banks, to reach these smaller businesses, as they cannot directly engage with them.
*   **Brexit and Fisheries:** The agreement between the EU and Norway is mentioned as not being beneficial for Ireland's fishing community, with a call for fair systems for fishermen. Separately, in the context of EU-UK relations, the joint management of fish stocks, particularly in the North Sea, is a critical point. There's a strong desire to ensure continued access to British waters for EU fishermen after 2026, avoiding further quota reductions.
*   **Agricultural Trade and Sanitary/Phytosanitary (SPS) Measures:** The trade relationship between the UK and the EU, specifically for agricultural products, is a point of concern. For Flanders, exporting EUR 4.8 billion worth of agricultural products to the UK annually, the current procedures, controls, and differing regulations are causing delays and uncertainty. There's a call for a workable agreement that facilitates trade without extra burdens or compromising quality, emphasizing less bureaucracy, more predictability, and fair market access for farmers.
*   **EU-Norway Agreement:** An agreement between the EU and Norway is mentioned, with a representative from Ireland stating it is not good for the Irish fishing community and expressing shame if any Irish MEP votes in its favor.
*   **Cohesion Policy and Agriculture:** One speaker mentions that while the main principles of cohesion policy (modernization, evolution, not revolution) are agreed upon, the practical implementation needs discussion. There's a call for more decentralization of cohesion and common agricultural policy to regional and local authorities, arguing that decisions made closer to citizens are better adapted to local realities and more efficient. The speaker also links cohesion policy investments to strengthening trust in the European project and combating Euroscepticism.

In essence, the discussion highlights efforts to support farmers, ensure fair trade, address the impact of international agreements on fishing communities, and integrate agricultural considerations within broader economic and cohesion policies.

Bu fena değil, ama modele daha fazla rehberlik vererek daha kapsamlı bir istem yazarak daha iyisini yapabiliriz.

Bunu yapan ilk kişiler biz değilmişiz ve LangChain'in bizim için önceden hazırlanmış istem kütüphanesi var.

👉 Aşağıdaki hücreyi çalıştırın ve nasıl çalıştığını anlamaya çalışın. (LangSmithMissingAPIKeyWarning hakkında bir uyarı alacaksınız, bunu görmezden gelebilirsiniz.)

In [25]:
from langchain_classic import hub

prompt_template = hub.pull("rlm/rag-prompt")

example_messages = prompt_template.invoke(
    {"context": "(context goes here)", "question": "(question goes here)"}
).to_messages()

print("\n")
print(example_messages[0].content)



You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: (question goes here) 
Context: (context goes here) 
Answer:


LangChain'in bizim için nasıl daha kesin bir istem oluşturduğunu görüyor musunuz? Bunu RAG'ımız için kullanalım!

👉 İlk olarak, tüm alınan belgeleri iki yeni satırla ayrılmış tek bir uzun dizgiye birleştirin.

In [26]:
docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)
Markdown(docs_content)

every applicant, whether in the private sector or in the public sector, to bring about a project 
which really gives a return to European taxpayers.
I noticed very well the remarks on small- and medium-sized enterprises, but also micro 
businesses, and I fully agree the access to credit for these companies, these very small companies, 
who are so important when it comes to the labour market inside the EU, is still an issue we really 
have to worry about and work on, and that's what we are doing as the EIB group. We cannot do 
this directly with SMEs and micro businesses in Europe. We always go through a financial 
intermediary, mostly European commercial banks – a very important element of our business.
I listened very carefully to the remarks on agriculture, and especially young farmers receive our 
attention when it comes to the area of agriculture. For this year, we envisage to invest at least 
EUR 3 billion in this area.
In the area of housing, which was also mentioned by honourable Members, we are trying to 
leverage the financing we are going to make available to a couple of billion euro, hopefully in a 
couple of years, to EUR 300 billion annually. We have three priorities in the area of housing: one – 
innovation, supporting innovative building technologies like modular housing to make 
construction faster, cheaper and easier; second – sustainability, scaling up energy efficient 
renovation to reduce living costs when it comes to energy prices; and three – affordability, 
strengthening support for public investment tailored to the specific needs of each country and 
piloting private investments.
Now, on the issue of climate, which is also close to a bit more than half of what we are doing 
annually. This is about climate adaptation; this is about dealing with droughts, it is about dealing 
with floods – we have seen both inside many countries of the European Union, and they require

dicho nada del hundimiento del barco ni de los más de mil asesinados –cooperantes, médicos y 
enfermeras– que intentan salvar vidas. Esas muertes son una mancha indeleble en la conciencia de 
los líderes europeos que siguen cooperando con el genocidio en Gaza.
Nuestra obligación como ciudadanos es movilizarnos como garantes del Derecho internacional y 
recordar que, cuando la barbarie se normaliza, la desobediencia es una obligación moral.
1-0232-0000
Kathleen Funchion (The Left). – A Uachtaráin, ba mhaith liom an comhaontú idir an tAontas 
Eorpach agus an Iorua a ardú. Níl an comhaontú seo go maith do phobal iascaireachta na 
hÉireann agus is mór an náire é aon Fheisire as Éirinn vótáil ina fhabhar. Tá ár n-iascairí i dteideal 
córas cothrom agus ba chóir dúinn oibriú ina threo sin. Táimid i Sinn Féin tiomanta chun seasamh 
suas ar son ár bpobal cois cósta agus iascaireachta chun a chinntiú go mbeidh todhchaí bhreá agus 
gheal acu.
1-0233-0000
Γεώργιος Αυτιάς (PPE). – Κυρία Πρόεδρε, κύριε Επίτροπε, η άσκηση οικονομικής πολιτικής, πέραν της 
ανταγωνιστικότητας και της σταθερότητας —και το ξέρετε πολύ καλά αυτό, γιατί η πατρίδα μου πέρασε 
από τρία μνημόνια— πρέπει να έχει και έντονο κοινωνικό χαρακτήρα, δηλαδή στήριξη μισθών και 
συντάξεων, στήριξη φορολογικών ελαφρύνσεων, λύση δημογραφικού, στέγη. Το ξέρετε πολύ καλά, 
κύριε Επίτροπε, το θέμα, και εσείς, αξιότιμοι συνάδελφοι. Μείωση της ανεργίας και φθηνή ενέργεια.
102 05-05-2025
Προς αυτή την κατεύθυνση, η χώρα μου κινείται με ταχύτατο ρυθμό, αποπληρώνει δάνεια δεκαετίες 
μπροστά, έχει άριστες κριτικές από οίκους αξιολόγησης και, παράλληλα, πλεόνασμα. Αυτό το 
πλεόνασμα, λοιπόν, επιστρέφεται στην κοινωνία.
Να ξέρετε, κύριε Επίτροπε, ότι αυτός ο βηματισμός θα συνεχιστεί και το επόμενο χρονικό διάστημα και 
προς αυτή την κατεύθυνση σας ενημερώνω συνεχώς.
1-0234-0000
Juan Fernando López Aguilar (S&D). – Señora presidenta, vicepresidente Fitto, usted estuvo

Koninkrijk en de Europese Unie is de Overeenkomst inzake sanitaire en fytosanitaire maatregelen 
(SPS-Overeenkomst) over voedselveiligheid en grenscontroles op levensmiddelen een van de 
belangrijkste dossiers. Mijn land Vlaanderen voert jaarlijks voor 4,8 miljard EUR aan 
landbouwproducten uit naar het Verenigd Koninkrijk. Deze handel wordt echter vertraagd door 
omslachtige procedures, extra controles en uiteenlopende regels, met alle gevolgen van dien. Onze 
boeren en bedrijven verdienen geen onzekerheid, maar duidelijke afspraken. Daarom vraag ik: 
stop met treuzelen, stop met eindeloos uitstellen. We hebben een werkbaar akkoord nodig dat 
handel mogelijk maakt, zonder extra lasten en zonder toegevingen op het gebied van kwaliteit. 
We hebben behoefte aan minder bureaucratie, meer voorspelbaarheid en meer handel. Onze 
boeren hebben recht op een vlotte export en eerlijke toegang tot de markt.
1-0101-0000
Bert-Jan Ruissen (ECR). – Voorzitter, "Beter een goede buur dan een verre vriend" – dat is de 
benadering die we moeten volgen in onze betrekkingen met het Verenigd Koninkrijk. De top van 
19 mei biedt een mooie kans om hier invulling aan te geven. Het is belangrijk om het "chagrijn" 
over de Brexit achter ons te laten. Laten we investeren in onze onderlinge relaties, de handel 
bevorderen en gezamenlijk inspanningen leveren rond defensie. Dit is absoluut geen luxe in deze 
tijden van geopolitieke spanningen.
Een belangrijk aandachtspunt is het gezamenlijk beheer van de visbestanden, met name in de 
Noordzee. Voor onze vissers is het daarbij van cruciaal belang om ook na 2026 de toegang tot de 
Britse wateren te behouden. We moeten voorkomen dat zij opnieuw quota moeten inleveren. Dit 
onderwerp verdient alle aandacht, ook tijdens de komende top. Als goede buren moeten we toch 
in staat zijn om hier goede afspraken over te maken.
1-0102-0000
Nathalie Loiseau (Renew). – Madam President, dear British friends. The EU-UK summit gives us

wzmacniać mosty, modernizować drogi dojazdowe czy budować je w takich parametrach, żeby 
mogły również służyć celom obronnym. I to nie jest militaryzowanie polityki spójności, ale danie 
możliwości tym regionom, które czują taką potrzebę, realizowania tych celów.
Szanowni państwo, panie komisarzu, bardzo serdecznie dziękuję za te dzisiejsze wystąpienia. 
Dziękuję za współpracę. Mam głębokie przekonanie, że ten dokument, który w czwartek 
przegłosujemy, również pomoże panu, bowiem znamy pana historię zawodową. Wiemy, że jest 
pan samorządowcem. Był pan szefem regionu, ministrem odpowiedzialnym również za politykę 
regionalną, więc wiemy, że rozumie pan potrzeby regionu, potrzeby społeczności lokalnych. Ale 
u nas w Polsce się mówi, że diabeł tkwi w szczegółach. Co do głównych założeń polityki 
spójności zgadzamy się również, że trzeba iść w kierunku modernizacji, ewolucji, nie rewolucji. 
Ale będziemy dyskutować na temat tego, jak to w praktyce ma wyglądać i jak Komisja Europejska 
to widzi. Mam nadzieję, że wspólnie osiągniemy sukces.
1-0214-0000
Presidente. – La discussione è chiusa.
La votazione si svolgerà giovedì.
Dichiarazioni scritte (articolo 178)
1-0214-5000
Dan-Ştefan Motreanu (PPE), în scris. – Doresc să transmit un mesaj clar Comisiei Europene: nu 
avem nevoie de o centralizare a politicii de coeziune și a politicii agricole comune, după modelul 
NextGenerationEU. Dimpotrivă, este nevoie de mai multă descentralizare către autoritățile 
regionale și locale, pentru că deciziile luate mai aproape de cetățeni sunt mai bine adaptate 
realităților din teren, mai eficiente și rapide.
Într-un climat geopolitic tot mai tensionat, reamintesc că politica de coeziune este cel mai vizibil și 
concret instrument al UE pentru majoritatea cetățenilor. Investițiile în spitale, infrastructură, 
educație și IMM-uri consolidează încrederea în proiectul european și combat eficient discursurile 
eurosceptice.
96 05-05-2025

👉 Sonra, sorgunuz ve alınan belgelerden başlayarak bir `prompt` oluşturun. Yukarıdaki örneğe bakmayı unutmayın.

In [27]:
prompt = prompt_template.invoke(
    {"context": docs_content, "question": query}
)

👉 Son olarak az önce oluşturduğumuz `the_prompt` ile LLM modelini kullanın:

In [28]:
answer = llm.invoke(prompt)

In [29]:
Markdown(answer.content)

The discussion on agricultural policy highlights a focus on young farmers, with plans to invest at least EUR 3 billion in this sector this year. There is also a concern about the impact of trade agreements, specifically the EU-Norway agreement on fisheries, which is seen as detrimental to Irish fishing communities. Additionally, there's a call for a more decentralized approach to agricultural policy, with decisions made closer to local realities for greater effectiveness.

🎉 İlk RAG'ımızı tamamladık: LLM kendisine sağladığımız belgelerde ***temelli*** metin üretti.

## 💾 Gömmelerimizi kalıcı hale getir

Şimdiye kadar bellekte vektör deposuyla çalıştık. Bu yüzden not defterinizi kapattığınızda, tüm gömmeleri de kaybedeceksiniz.

⚠️ Bu gömmelerin sağlayıcınızın platformunda, bu durumda Google'ın makinelerinde çalışan modeller tarafından üretildiğini unutmayın. Ve bedava çalışmazlar. 💰

Bunun gibi bir, nispeten küçük belge için maliyet düşüktür, ama hızla artar. Şimdiye kadar sadece bir günün transkriptleriyle çalıştık. Oturum başına 3 tane daha, yılda 12 oturum, birden fazla yıl var...

Bunu çözmek için sadece vektör depomuzla kalıcı bir taneyi değiştireceğiz. Bu LangChain'in avantajıdır: bileşenleri değiştirmek çok kolay.

Bellekteki vektör depomuz deneme için harikaydı, şimdi başka bir taneyle değiştireceğiz. Çok popüler bir vektör deposu olan [Chroma](https://www.trychroma.com/)'yı kullanacağız. Bunu yerel olarak çalıştırabilir ve LangChain aracılığıyla kullanabiliriz.

Tüm akışımızı yeniden oluşturacağız. Her şeyi birkaç kod hücresinde tekrar bir araya getirmeye çalışmak iyi bir alıştırmadır. Aynı zamanda her şeyi yeniden kullanılabilir koda dönüştüreceğiz.

Sonunda iki fonksiyon istiyoruz:

1. `embed_and_store()`: Başka bir oturumun transkriptini vektör veritabanımıza ekle, böylece alacağımız daha fazla veri olsun.
2. `answer()`: Vektör depomuzla farklı sorularla sorgula.

#### 1. Bir Chroma vektör deposu başlat

👉 **Veri kalıcılığıyla** (yani verileri diskteki bir dizinde saklayarak) Chroma vektör deposunun nasıl oluşturulacağını görmek için [LangChain'in belgelerine](https://python.langchain.com/docs/integrations/vectorstores/chroma/) bakın.

In [30]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="ep_plenary",
    embedding_function=embeddings,
    persist_directory="./chroma_ep_follower",
)

#### 2. `embed_and_store()` oluştur

👉 Bu fonksiyon için kodu tamamlayın:

In [31]:
def embed_and_store(file_path, vector_store):
    """Load a PDF file, split it into chunks, and store the chunks in a vector store."""
    # Load the PDF file
    loader = PyPDFLoader(file_path, mode='single')
    pdf = loader.load()

    # Split the pages into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
        is_separator_regex=False,
    )
    all_splits = text_splitter.split_documents(pdf)

    # # Add the session_date to the metadata
    # for split in all_splits:
    #     split.metadata['session_date'] = session_date

    # Add the chunks to the vector store
    document_ids = vector_store.add_documents(all_splits)
    print(f"Added {len(document_ids)} documents from {os.path.basename(file_path)}")

    return document_ids

👉 Fonksiyonunuzu bir dosya veya hatta iki dosyayla deneyin:

In [32]:
file_path = "data/CRE-10-2025-05-05_EN.pdf"

document_ids = embed_and_store(file_path, vector_store)

Added 429 documents from CRE-10-2025-05-05_EN.pdf


In [33]:
vector_store.get_by_ids(document_ids[:3])

[Document(id='e53595b1-8429-45e4-b223-46b6fc8d1cf8', metadata={'producer': 'Aspose.Words for Java 24.1.0', 'total_pages': 106, 'source': 'data/CRE-10-2025-05-05_EN.pdf', 'creationdate': '', 'creator': 'Aspose.Words'}, page_content='2024-2029\nПЪЛЕН ПРОТОКОЛ НА РАЗИСКВАНИЯТА DEBAŠU STENOGRAMMA\nACTA LITERAL DE LOS DEBATES POSĖDŽIO STENOGRAMA\nDOSLOVNÝ ZÁZNAM ZE ZASEDÁNÍ AZ ÜLÉSEK SZÓ SZERINTI JEGYZŐKÖNYVE\nFULDSTÆNDIGT FORHANDLINGSREFERAT RAPPORTI VERBATIM TAD-DIBATTITI\nAUSFÜHRLICHE SITZUNGSBERICHTE VOLLEDIG VERSLAG VAN DE VERGADERINGEN\nISTUNGI STENOGRAMM PEŁNE SPRAWOZDANIE Z OBRAD\nΠΛΗΡΗ ΠΡΑΚΤΙΚΑ ΤΩΝ ΣΥΖΗΤΗΣΕΩΝ RELATO INTEGRAL DOS DEBATES\nVERBATIM REPORT OF PROCEEDINGS STENOGRAMA DEZBATERILOR\nCOMPTE RENDU IN EXTENSO DES DÉBATS DOSLOVNÝ ZÁPIS Z ROZPRÁV\nTUARASCÁIL FOCAL AR FHOCAL NA N-IMEACHTAÍ DOBESEDNI ZAPISI RAZPRAV\nDOSLOVNO IZVJEŠĆE SANATARKAT ISTUNTOSELOSTUKSET\nRESOCONTO INTEGRALE DELLE DISCUSSIONI FULLSTÄNDIGT FÖRHANDLINGSREFERAT\nПонеделник - lunes - Pondělí - mandag - Mont

#### 3. `answer()` oluştur

👉 Bu fonksiyon için kodu tamamlayın:

In [34]:
def answer(query, vector_store, llm, prompt_template=None):
    """Answer a query using the vector store and the language model."""
    # Retrieve similar documents from the vector store
    retrieved_docs = vector_store.similarity_search(query, k=6)

    # Create the prompt
    docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)

    # If no prompt template is provided, use the default one
    if not prompt_template:
        prompt_template = hub.pull("rlm/rag-prompt")

    prompt = prompt_template.invoke(
        {"context": docs_content, "question": query}
    )

    # Get the answer from the language model
    answer = llm.invoke(prompt)

    return answer.content

👉 Fonksiyonunuzu beğendiğiniz bir sorguyla deneyin:

In [35]:
query = "What is being said about international trade?"
Markdown(answer(query, vector_store, llm, prompt_template))

The Trade and Cooperation Agreement (TCA) between the EU and the UK is described as the most ambitious free trade agreement the EU has concluded with any third country, responding to the UK's red lines. However, there is a desire to further exploit its potential and deepen cooperation in areas such as finance, SPS, emissions trading, and youth mobility. The upcoming EU-UK summit is seen as an opportunity to strengthen this relationship.

🏁 Tebrikler! Artık LangChain kullanarak RAG'da ustalaştınız ve vektör deponuza daha fazla belge eklemek ve onu sorgulamak için yeniden kullanılabilir fonksiyonlar yapmayı öğrendiniz.

## [İsteğe Bağlı] Meta veri ekleme

Kurduğumuz RAG, vektör deposundaki tüm belgeleri sorgular. Orada birden fazla yılın bilgisinin olduğunu düşünün. Yıllara veya tarihlere göre filtreyebilsek kullanışlı olurdu, değil mi?

Bunu nasıl yaparız? Vektör deposundaki belgelerin meta veri içerdiğini unutmayın. Eğer tarihi ekleyebilseydik, daha sonra filtrelemek için kullanabilirdik.

İpucu: Meta verilerinizi pipeline'ınızda olabildiğince erken ekleyin. Verileriniz vektör deposunda saklandıktan sonra eklemeye çalışmayın.

👉 `embed_and_store()` fonksiyonunuzu uyarlayın.

In [36]:
def embed_and_store_fancy(file_path, vector_store, session_date):
    """Load a PDF file, split it into chunks, and store the chunks in a vector store.
    Session_date is added to the metadata of each chunk."""
    loader = PyPDFLoader(file_path, mode="single")
    pdf = loader.load()

    for doc in pdf:
        doc.metadata["session_date"] = session_date

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
        is_separator_regex=False,
    )
    all_splits = text_splitter.split_documents(pdf)

    document_ids = vector_store.add_documents(all_splits)
    print(f"Added {len(document_ids)} documents from {os.path.basename(file_path)}")

    return document_ids

👉 Fonksiyonunuzu deneyin ve vektör deponuzun ek meta veri içerdiğini kontrol edin.

In [37]:
file_path = "data/CRE-10-2025-05-05_EN.pdf"
document_ids = embed_and_store_fancy(file_path, vector_store, "2025-05-05")

Added 429 documents from CRE-10-2025-05-05_EN.pdf


Şimdi alıcıyı kullanıcının sorduğu tarihe göre sınırlamamız gerekiyor.

👉 `answer()` fonksiyonunuzu bir tarih alabilecek ve yeni meta verilere dayalı olarak belgeleri filtreleyebilecek şekilde uyarlayın.

In [50]:
def answer_fancy(query, vector_store, llm, prompt_template=None, session_date=None, session_year=None, session_month=None):
    """Answer a query using the vector store and the language model."""
  
    if session_date:
        filter = {"session_date": session_date}
    elif session_month:
        filter = {"month": session_month}
    elif session_month:
        filter = {"year": session_year}
    else:
        filter = {}

    retrieved_docs = vector_store.similarity_search(query, k=6, filter=filter)


    if not retrieved_docs:
        return "No documents found for the given filters."


    docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)


    if not prompt_template:
        prompt_template = hub.pull("rlm/rag-prompt")

    prompt = prompt_template.invoke(
        {"context": docs_content, "question": query}
    )

    answer = llm.invoke(prompt)

    return answer.content

In [52]:
query = "Summarize the discussion on agricultural policy."
Markdown(answer_fancy(query, vector_store, llm, prompt_template, session_date="2025-05-05"))

The discussion on agricultural policy highlighted concerns about the centralization of the Common Agricultural Policy, with a call for more decentralization to regional and local authorities. There was also a mention of the need for a workable agreement to facilitate trade in agricultural products without additional burdens or concessions on quality, aiming for less bureaucracy and more predictability. Finally, a significant investment of at least EUR 3 billion is planned for young farmers, indicating a focus on supporting this sector.

In [53]:
Markdown(answer_fancy(query, vector_store, llm, prompt_template, session_date="2025-05-06"))

No documents found for the given filters.

Harika! Güçlü bir RAG sistemi oluşturmak için benzerlik aramasını meta veri aramasıyla birleştirdiniz!